In [5]:
pip install torch torchvision

Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
# تبدیل داده‌ها به تانسور و نرمال‌سازی
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# دانلود و بارگذاری داده‌های آموزشی و تست
trainset = torchvision.datasets.CIFAR10(root="D:\\_frzna_\\Projects\\PycharmProjects\\CNN\\cifar-10-batches-py", train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

100%|███████████████████████████████████████████████████████████████████████████████| 170M/170M [02:10<00:00, 1.31MB/s]
100%|███████████████████████████████████████████████████████████████████████████████| 170M/170M [01:42<00:00, 1.67MB/s]


In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # لایه کانولوشن اول
        self.conv1 = nn.Conv2d(3, 32, 3)  # ورودی 3 کانال (RGB)، 32 فیلتر، کرنل 3x3
        self.pool = nn.MaxPool2d(2, 2)  # Max Pooling با اندازه 2x2

        # لایه کانولوشن دوم
        self.conv2 = nn.Conv2d(32, 64, 3)  # ورودی 32، خروجی 64 فیلتر

        # لایه‌های Fully Connected
        self.fc1 = nn.Linear(64 * 6 * 6, 128)  # بعد از دو لایه Pooling، ابعاد 6x6 می‌شود
        self.fc2 = nn.Linear(128, 10)  # خروجی 10 نورون (10 کلاس)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # Conv1 → ReLU → Pooling
        x = self.pool(F.relu(self.conv2(x)))  # Conv2 → ReLU → Pooling
        x = torch.flatten(x, 1)  # Flatten
        x = F.relu(self.fc1(x))  # FC1 → ReLU
        x = self.fc2(x)  # FC2 (خروجی)
        return x

In [11]:
model = CNN()

criterion = nn.CrossEntropyLoss()  # برای طبقه‌بندی چندکلاسه
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):  # 10 دوره آموزش
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()  # پاک کردن گرادیان‌های قبلی

        outputs = model(inputs)  # پیش‌بینی مدل
        loss = criterion(outputs, labels)  # محاسبه خطا
        loss.backward()  # انتشار معکوس
        optimizer.step()  # به‌روزرسانی وزن‌ها

        running_loss += loss.item()

    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(trainloader)}')

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Accuracy: {100 * correct / total}% in {total} images')

Epoch 1, Loss: 1.412473019126736
Epoch 2, Loss: 1.0414021530419664
Epoch 3, Loss: 0.8871514912304062
Epoch 4, Loss: 0.7716826744320447
Epoch 5, Loss: 0.6751528580094237
Epoch 6, Loss: 0.592101705539257
Epoch 7, Loss: 0.5118970723293931
Epoch 8, Loss: 0.44109894948847156
Epoch 9, Loss: 0.3722288070241814
Epoch 10, Loss: 0.3221115250607281
Test Accuracy: 71.79% in 10000 images
